In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("IcebergNessieMinIO")
    .config(
        "spark.jars.packages",
        ",".join([
            "org.apache.iceberg:iceberg-spark-runtime-3.4_2.12:1.6.1",
            "org.projectnessie.nessie-integrations:nessie-spark-extensions-3.3_2.12:0.67.0",
            "org.apache.hadoop:hadoop-aws:3.3.4"
        ])
    )
    # ---- Catálogo Nessie ----
    .config("spark.sql.catalog.demo", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.demo.catalog-impl", "org.apache.iceberg.nessie.NessieCatalog")
    .config("spark.sql.catalog.demo.uri", "http://localhost:19120/api/v1")
    .config("spark.sql.catalog.demo.ref", "main")
    .config("spark.sql.catalog.demo.warehouse", "s3a://bronze/warehouse/")
    .config("spark.sql.catalog.demo.authentication.type", "NONE")

    # ---- MinIO ----
    .config("spark.hadoop.fs.s3a.endpoint", "http://localhost:9000")
    .config("spark.hadoop.fs.s3a.access.key", "teste")
    .config("spark.hadoop.fs.s3a.secret.key", "password")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

    # ---- Iceberg extensions ----
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
    .getOrCreate()
)


your 131072x1 screen size is bogus. expect trouble
25/10/19 21:05:00 WARN Utils: Your hostname, DESKTOP-4LPUPP7 resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/10/19 21:05:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/andre/miniforge3/envs/iceberg/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/andre/.ivy2/cache
The jars for the packages stored in: /home/andre/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.4_2.12 added as a dependency
org.projectnessie.nessie-integrations#nessie-spark-extensions-3.3_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5137db43-e65b-4108-ab49-257214a48ee6;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.4_2.12;1.6.1 in central
	found org.projectnessie.nessie-integrations#nessie-spark-extensions-3.3_2.12;0.67.0 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 187ms :: artifacts dl 8ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in

In [6]:
spark.sql("CREATE DATABASE IF NOT EXISTS demo.db_nessie")

DataFrame[]

In [7]:
spark.sql("""
    CREATE TABLE demo.db_nessie.teste (
        id INT,
        nome STRING,
        valor DOUBLE
    ) USING ICEBERG
""")

print("✅ Tabela criada com sucesso!")

✅ Tabela criada com sucesso!


In [8]:
spark.sql("INSERT INTO demo.db_nessie.teste VALUES (1, 'A', 10.5), (2, 'B', 20.0)")

DataFrame[]

In [9]:
spark.sql("SELECT * FROM demo.db_nessie.teste").show()

+---+----+-----+
| id|nome|valor|
+---+----+-----+
|  1|   A| 10.5|
|  2|   B| 20.0|
+---+----+-----+



In [10]:
spark.sql("INSERT INTO demo.db_nessie.teste VALUES (3, 'C', 11.5), (4, 'B', 21.0)")

DataFrame[]

In [11]:
spark.sql("SELECT * FROM demo.db_nessie.teste").show()

+---+----+-----+
| id|nome|valor|
+---+----+-----+
|  1|   A| 10.5|
|  2|   B| 20.0|
|  3|   C| 11.5|
|  4|   B| 21.0|
+---+----+-----+

